# Visa Approvals  (2017): Part 2 - expanded new data source

## Preprocessing and Wrangling 2
  
  
**AMOD-5410H: Big Data**   
**Winter 2018**  
**Nicholas Hopewell - 0496633**

The original source of these data comes from the US department of labour. These data are come from an updated data source containing the same columns and meta data as the first data set but with more features. These data are only from 2017 and include multiple visa types, not only H1b.

In [240]:
import pandas as pd
import numpy as np
# to avoid scientific notation
pd.options.display.float_format = '{:.2f}'.format
# filte out package warnings
import warnings
warnings.filterwarnings('ignore')

In [241]:
# read data
visa_data = pd.read_csv("C:\\Users\\nicho\\Desktop\\big data course proj\\H-1B data\\H-1B_Disclosure_Data_FY17.csv")

In [243]:
# list col names
list(visa_data.columns)

['Unnamed: 0',
 'CASE_NUMBER',
 'CASE_STATUS',
 'CASE_SUBMITTED',
 'DECISION_DATE',
 'VISA_CLASS',
 'EMPLOYMENT_START_DATE',
 'EMPLOYMENT_END_DATE',
 'EMPLOYER_NAME',
 'EMPLOYER_BUSINESS_DBA',
 'EMPLOYER_ADDRESS',
 'EMPLOYER_CITY',
 'EMPLOYER_STATE',
 'EMPLOYER_POSTAL_CODE',
 'EMPLOYER_COUNTRY',
 'EMPLOYER_PROVINCE',
 'EMPLOYER_PHONE',
 'EMPLOYER_PHONE_EXT',
 'AGENT_REPRESENTING_EMPLOYER',
 'AGENT_ATTORNEY_NAME',
 'AGENT_ATTORNEY_CITY',
 'AGENT_ATTORNEY_STATE',
 'JOB_TITLE',
 'SOC_CODE',
 'SOC_NAME',
 'NAICS_CODE',
 'TOTAL_WORKERS',
 'NEW_EMPLOYMENT',
 'CONTINUED_EMPLOYMENT',
 'CHANGE_PREVIOUS_EMPLOYMENT',
 'NEW_CONCURRENT_EMPLOYMENT',
 'CHANGE_EMPLOYER',
 'AMENDED_PETITION',
 'FULL_TIME_POSITION',
 'PREVAILING_WAGE',
 'PW_UNIT_OF_PAY',
 'PW_WAGE_LEVEL',
 'PW_SOURCE',
 'PW_SOURCE_YEAR',
 'PW_SOURCE_OTHER',
 'WAGE_RATE_OF_PAY_FROM',
 'WAGE_RATE_OF_PAY_TO',
 'WAGE_UNIT_OF_PAY',
 'H1B_DEPENDENT',
 'WILLFUL_VIOLATOR',
 'SUPPORT_H1B',
 'LABOR_CON_AGREE',
 'PUBLIC_DISCLOSURE_LOCATION',
 'WOR

There are many columns which provide no interesting information for ML. Also, the validity of the data is not sound of I include many different types of visas - the certification process and applicant pool is likely very different and not comparable with an identical model. 

The first thing I need to do is select h1b records only and then filter out columns I do not want. 

In [244]:
# subset only h1b
visa_data = visa_data[visa_data['VISA_CLASS'] == 'H-1B']
# interesting cols
keep_cols = ['CASE_STATUS', 'EMPLOYMENT_START_DATE', 'EMPLOYMENT_END_DATE',
             'EMPLOYER_STATE', 'AGENT_REPRESENTING_EMPLOYER', 'SOC_TITLE',
             'TOTAL_WORKERS', 'NEW_EMPLOYMENT', 'CONTINUED_EMPLOYMENT',
             'CHANGE_PREVIOUS_EMPLOYMENT', 'NEW_CONCURRENT_EMPLOYMENT', 'CHANGE_EMPLOYER',
             'AMENDED_PETITION', 'FULL_TIME_POSITION', 'PREVAILING_WAGE', 
             'PW_UNIT_OF_PAY', 'PW_WAGE_LEVEL', 'PW_SOURCE', 
             'WAGE_RATE_OF_PAY_FROM', 'WAGE_RATE_OF_PAY_TO', 'H1B_DEPENDENT',
             'WILLFUL_VIOLATOR', 'SUPPORT_H1B', 'LABOR_CON_AGREE',
             'WORKSITE_STATE']
# filter interesting cols
visa_data = visa_data.filter(items = keep_cols)

In [245]:
visa_data.head(5)

,CASE_STATUS,EMPLOYMENT_START_DATE,EMPLOYMENT_END_DATE,EMPLOYER_STATE,AGENT_REPRESENTING_EMPLOYER,TOTAL_WORKERS,NEW_EMPLOYMENT,CONTINUED_EMPLOYMENT,CHANGE_PREVIOUS_EMPLOYMENT,NEW_CONCURRENT_EMPLOYMENT,...,PW_UNIT_OF_PAY,PW_WAGE_LEVEL,PW_SOURCE,WAGE_RATE_OF_PAY_FROM,WAGE_RATE_OF_PAY_TO,H1B_DEPENDENT,WILLFUL_VIOLATOR,SUPPORT_H1B,LABOR_CON_AGREE,WORKSITE_STATE
0,CERTIFIED-WITHDRAWN,2016-08-10,2019-08-10,IL,Y,1,1,0,0,0,...,Year,Level I,OES,65811.00,67320.00,N,N,NaN,Y,IL
1,CERTIFIED-WITHDRAWN,2016-08-16,2019-08-16,IL,Y,1,1,0,0,0,...,Year,NaN,Other,53000.00,57200.00,N,N,NaN,Y,IL
2,CERTIFIED-WITHDRAWN,2016-09-09,2019-09-09,DC,Y,2,2,0,0,0,...,Year,Level II,OES,77000.00,0.00,Y,N,Y,NaN,DC
3,WITHDRAWN,2017-01-26,2020-01-25,MI,N,1,1,0,0,0,...,Year,Level III,OES,102000.00,0.00,Y,N,Y,NaN,NJ
4,CERTIFIED-WITHDRAWN,2015-03-01,2018-03-01,NC,Y,1,0,0,0,0,...,Year,Level III,OES,132500.00,0.00,N,N,NaN,Y,NY


In [246]:
# rows and cols
visa_data.shape

(610304, 24)

This data set includes a much smaller number of rows (it only includes one year), so I have to be much, much more careful about the records I strip. Making a statistically significant impact on the model by stripping n records is much easier now. 

### Data Dictionary: 

I will immediately convert prevailing wage into yearly units across the board for consistency.

In [247]:
visa_data.isnull().sum()

CASE_STATUS                         0
EMPLOYMENT_START_DATE              24
EMPLOYMENT_END_DATE                24
EMPLOYER_STATE                     18
AGENT_REPRESENTING_EMPLOYER     92941
TOTAL_WORKERS                       0
NEW_EMPLOYMENT                      0
CONTINUED_EMPLOYMENT                0
CHANGE_PREVIOUS_EMPLOYMENT          0
NEW_CONCURRENT_EMPLOYMENT           0
CHANGE_EMPLOYER                     0
AMENDED_PETITION                    0
FULL_TIME_POSITION                  5
PREVAILING_WAGE                     1
PW_UNIT_OF_PAY                     32
PW_WAGE_LEVEL                  119458
PW_SOURCE                          31
WAGE_RATE_OF_PAY_FROM               0
WAGE_RATE_OF_PAY_TO                 1
H1B_DEPENDENT                       4
WILLFUL_VIOLATOR                    4
SUPPORT_H1B                    369872
LABOR_CON_AGREE                365720
WORKSITE_STATE                      9
dtype: int64

In [248]:
# to convert units of pay
def pay_to_yearly(wage, unit):
    if unit == "Year":
        result = wage
    elif unit   == "Week":
        result = 52 * wage
    elif unit   == "Month":
        result = 12 * wage
    elif unit == "Hour":
        result = 2080 * wage
    else :
        result = 26 * wage
    return result
    

In [249]:
# drop if unit unknown
visa_data = visa_data.dropna(subset=['PW_UNIT_OF_PAY'], how='all')
# apply function to wage
visa_data['PREVAILING_WAGE'] = visa_data[['PREVAILING_WAGE', 'PW_UNIT_OF_PAY']].apply(
    lambda x: pay_to_yearly(*x), axis = 1
)

In [250]:
visa_data = visa_data.drop(['PW_UNIT_OF_PAY', 'AGENT_REPRESENTING_EMPLOYER', 
                            'PW_WAGE_LEVEL', 'SUPPORT_H1B', 'LABOR_CON_AGREE'], axis=1)

In [251]:
visa_data.isnull().sum()

CASE_STATUS                    0
EMPLOYMENT_START_DATE         22
EMPLOYMENT_END_DATE           22
EMPLOYER_STATE                17
TOTAL_WORKERS                  0
NEW_EMPLOYMENT                 0
CONTINUED_EMPLOYMENT           0
CHANGE_PREVIOUS_EMPLOYMENT     0
NEW_CONCURRENT_EMPLOYMENT      0
CHANGE_EMPLOYER                0
AMENDED_PETITION               0
FULL_TIME_POSITION             4
PREVAILING_WAGE                0
PW_SOURCE                      4
WAGE_RATE_OF_PAY_FROM          0
WAGE_RATE_OF_PAY_TO            1
H1B_DEPENDENT                  2
WILLFUL_VIOLATOR               2
WORKSITE_STATE                 5
dtype: int64

In [252]:
visa_data = visa_data.dropna()

In [253]:
visa_data.shape

(610216, 19)

This data set is fine for analyzing and I will keep it in this state to analyze. I will drop some more columns for modelling. 

In [269]:
final_visa_data = visa_data.drop(['EMPLOYMENT_START_DATE', 'EMPLOYMENT_END_DATE', 
                                  'EMPLOYER_STATE', 'WORKSITE_STATE'], axis=1)

In [270]:
final_visa_data.head(5)

,CASE_STATUS,TOTAL_WORKERS,NEW_EMPLOYMENT,CONTINUED_EMPLOYMENT,CHANGE_PREVIOUS_EMPLOYMENT,NEW_CONCURRENT_EMPLOYMENT,CHANGE_EMPLOYER,AMENDED_PETITION,FULL_TIME_POSITION,PREVAILING_WAGE,PW_SOURCE,WAGE_RATE_OF_PAY_FROM,WAGE_RATE_OF_PAY_TO,H1B_DEPENDENT,WILLFUL_VIOLATOR
0,CERTIFIED-WITHDRAWN,1,1,0,0,0,0,0,Y,59197.00,OES,65811.00,67320.00,N,N
1,CERTIFIED-WITHDRAWN,1,1,0,0,0,0,0,Y,49800.00,Other,53000.00,57200.00,N,N
2,CERTIFIED-WITHDRAWN,2,2,0,0,0,0,0,Y,76502.00,OES,77000.00,0.00,Y,N
3,WITHDRAWN,1,1,0,0,0,0,0,Y,90376.00,OES,102000.00,0.00,Y,N
4,CERTIFIED-WITHDRAWN,1,0,0,0,0,1,0,Y,116605.00,OES,132500.00,0.00,N,N


In [271]:
final_visa_data.dtypes

CASE_STATUS                    object
TOTAL_WORKERS                   int64
NEW_EMPLOYMENT                  int64
CONTINUED_EMPLOYMENT            int64
CHANGE_PREVIOUS_EMPLOYMENT      int64
NEW_CONCURRENT_EMPLOYMENT       int64
CHANGE_EMPLOYER                 int64
AMENDED_PETITION                int64
FULL_TIME_POSITION             object
PREVAILING_WAGE               float64
PW_SOURCE                      object
WAGE_RATE_OF_PAY_FROM         float64
WAGE_RATE_OF_PAY_TO           float64
H1B_DEPENDENT                  object
WILLFUL_VIOLATOR               object
dtype: object

Here are the levels of the outcome variable:

In [272]:
list(visa_data.CASE_STATUS.unique())

['CERTIFIED-WITHDRAWN', 'WITHDRAWN', 'CERTIFIED', 'DENIED']

The 'withdrawn' individuals are kind of unknown. I do now know if they were certified or denied.

In [273]:
len(final_visa_data[final_visa_data['CASE_STATUS'] == 'WITHDRAWN'].index)

20312

In [275]:
list(final_visa_data.columns.values)

['CASE_STATUS',
 'TOTAL_WORKERS',
 'NEW_EMPLOYMENT',
 'CONTINUED_EMPLOYMENT',
 'CHANGE_PREVIOUS_EMPLOYMENT',
 'NEW_CONCURRENT_EMPLOYMENT',
 'CHANGE_EMPLOYER',
 'AMENDED_PETITION',
 'FULL_TIME_POSITION',
 'PREVAILING_WAGE',
 'PW_SOURCE',
 'WAGE_RATE_OF_PAY_FROM',
 'WAGE_RATE_OF_PAY_TO',
 'H1B_DEPENDENT',
 'WILLFUL_VIOLATOR']

In [277]:
final_visa_data =final_visa_data[final_visa_data['CASE_STATUS'] != 'WITHDRAWN']
len(final_visa_data[final_visa_data['CASE_STATUS'] == 'WITHDRAWN'].index)

0

Now I will change the 'certified-withdrawn' to simply 'certified' to make this a 2 class problem and because that third class is artificial and was not a decision made by the US government but rather the individual to withdraw. These individuals were, in fact, certified by the US government so it is okay to label them as simply certified. 

In [278]:
# overwrite val
final_visa_data.CASE_STATUS.replace('CERTIFIED-WITHDRAWN', 'CERTIFIED', inplace=True)

In [279]:
final_visa_data.shape

(589904, 15)

In [280]:
final_visa_data.head(5)

,CASE_STATUS,TOTAL_WORKERS,NEW_EMPLOYMENT,CONTINUED_EMPLOYMENT,CHANGE_PREVIOUS_EMPLOYMENT,NEW_CONCURRENT_EMPLOYMENT,CHANGE_EMPLOYER,AMENDED_PETITION,FULL_TIME_POSITION,PREVAILING_WAGE,PW_SOURCE,WAGE_RATE_OF_PAY_FROM,WAGE_RATE_OF_PAY_TO,H1B_DEPENDENT,WILLFUL_VIOLATOR
0,CERTIFIED,1,1,0,0,0,0,0,Y,59197.00,OES,65811.00,67320.00,N,N
1,CERTIFIED,1,1,0,0,0,0,0,Y,49800.00,Other,53000.00,57200.00,N,N
2,CERTIFIED,2,2,0,0,0,0,0,Y,76502.00,OES,77000.00,0.00,Y,N
4,CERTIFIED,1,0,0,0,0,1,0,Y,116605.00,OES,132500.00,0.00,N,N
5,CERTIFIED,1,1,0,0,0,0,0,Y,59405.00,OES,71750.00,0.00,N,N


One huge problem with financial values is missing data can be encoded as 0 or 0.00 etc...

In [281]:
len(final_visa_data[final_visa_data['WAGE_RATE_OF_PAY_TO'] == 0.00].index)

462225

We can see that in the majority of cases, a maximum proposed wage was not included. Having 0.00 in so many spots while not appearing as missing data will really impact model performance. Because of this, I will make this a binary factor with 2 levels (whether or not a max wage was proposed). 

In [282]:
final_visa_data['WAGE_RATE_OF_PAY_TO'] = pd.Series(np.where(final_visa_data.WAGE_RATE_OF_PAY_TO.values > 0.00, 'Y', 'N'),
                                              final_visa_data.index)

In [283]:
len(final_visa_data[final_visa_data['WAGE_RATE_OF_PAY_FROM'] == 0.00].index)

1

In [284]:
len(final_visa_data[final_visa_data['PREVAILING_WAGE'] == 0.00].index)

6

In [285]:
final_visa_data= final_visa_data[(final_visa_data['WAGE_RATE_OF_PAY_FROM'] != 0.00)
                               & (final_visa_data['PREVAILING_WAGE'] != 0.00)]

I will print out the levels of all object types to confirm nothing strange happened during data entry:

In [286]:
# Get levels of categorical data
def get_levels(df):
    for column in df:
        if df[column].dtype == 'O':
            print(f'{column}: {list(df[column].unique())}')

In [287]:
get_levels(final_visa_data)

CASE_STATUS: ['CERTIFIED', 'DENIED']
FULL_TIME_POSITION: ['Y', 'N']
PW_SOURCE: ['OES', 'Other', 'CBA', 'DBA', 'SCA']
WAGE_RATE_OF_PAY_TO: ['Y', 'N']
H1B_DEPENDENT: ['N', 'Y']
WILLFUL_VIOLATOR: ['N', 'Y']


Looks good.

Now I will look at the numeric columns to spot any potential issues:

In [288]:
final_visa_data.describe()

,TOTAL_WORKERS,NEW_EMPLOYMENT,CONTINUED_EMPLOYMENT,CHANGE_PREVIOUS_EMPLOYMENT,NEW_CONCURRENT_EMPLOYMENT,CHANGE_EMPLOYER,AMENDED_PETITION,PREVAILING_WAGE,WAGE_RATE_OF_PAY_FROM
count,589897.00,589897.00,589897.00,589897.00,589897.00,589897.00,589897.00,589897.00,589897.00
mean,1.93,0.84,0.33,0.13,0.01,0.27,0.34,94056.44,82460.47
std,4.92,3.62,1.26,0.97,0.16,1.11,1.45,1617300.36,41184.99
min,1.00,0.00,0.00,0.00,0.00,0.00,0.00,25.58,7.25
25%,1.00,0.00,0.00,0.00,0.00,0.00,0.00,59488.00,63000.00
50%,1.00,0.00,0.00,0.00,0.00,0.00,0.00,71677.00,75800.00
75%,1.00,1.00,0.00,0.00,0.00,0.00,0.00,89814.00,99287.59
max,155.00,150.00,55.00,50.00,50.00,150.00,50.00,401360960.00,1688000.00


Everything looks fine except for the prevailing wage max value. I explained this in detail in the first notebook - I had an issue with this column and explored it in detail. I will not go over the potential issues this could cause - see first notebook. 

See top 40 wages:

In [289]:
final_visa_data.nlargest(40, 'PREVAILING_WAGE')

,CASE_STATUS,TOTAL_WORKERS,NEW_EMPLOYMENT,CONTINUED_EMPLOYMENT,CHANGE_PREVIOUS_EMPLOYMENT,NEW_CONCURRENT_EMPLOYMENT,CHANGE_EMPLOYER,AMENDED_PETITION,FULL_TIME_POSITION,PREVAILING_WAGE,PW_SOURCE,WAGE_RATE_OF_PAY_FROM,WAGE_RATE_OF_PAY_TO,H1B_DEPENDENT,WILLFUL_VIOLATOR
596248,DENIED,1,0,0,0,0,1,0,Y,401360960.00,OES,212000.00,N,N,N
487435,DENIED,1,0,1,0,0,0,0,Y,297785280.00,OES,155000.00,Y,N,N
572984,DENIED,1,0,0,0,0,1,0,Y,247557440.00,Other,138000.00,N,N,N
222828,DENIED,3,3,0,0,0,0,0,Y,201622735.00,OES,124093.00,N,Y,N
559182,DENIED,1,0,0,0,0,0,1,Y,196634880.00,OES,94537.00,Y,N,N
52968,DENIED,1,0,0,0,0,1,0,Y,195813280.00,OES,106000.00,N,N,N
617864,DENIED,1,0,0,1,0,0,0,Y,183655680.00,OES,90000.00,N,Y,N
26034,DENIED,1,0,0,0,0,0,1,Y,181319840.00,OES,96187.00,N,N,N
82201,DENIED,1,0,1,0,0,0,0,Y,172839680.00,OES,85000.00,N,Y,N
165485,DENIED,1,1,0,0,0,0,0,Y,170849120.00,OES,100000.00,N,N,N


Notice how they were all denied their visa. Something is fishy with these data. 

Based on the above, and my exploration in the first notebook, I will drop above a threshold.

In [290]:
# drop records
final_visa_data = final_visa_data.drop(final_visa_data.index[final_visa_data.PREVAILING_WAGE > 5000000.00])

final_visa_data.nlargest(100, 'PREVAILING_WAGE')

The very final thing I will do right now is hot encode these data because I know MLlib's VectorAssembler does not accept string types. This is quite easy for this data set; most of the factors are yes or no.

Most established machine learning tools support nominal strings because the decision tree algorithm should work on nominal variables. In R these are called factors and in WEKA these are called nominals. With SciKit and MLlib, this nominal support simply is not there - this is actually an interesting issue as encoding can be done wrong and an unnatural ordering can be imposed on data which does no have an inherent ordering. Hot encoding works, but ordinal encoding does not (although it one might think it is best to encode yes and no ans 1 and 2). We absolutely do not want to impose an ordering to nominal data. 

In [291]:
cols = final_visa_data[['FULL_TIME_POSITION', 'PW_SOURCE', 
                        'WAGE_RATE_OF_PAY_TO', 'H1B_DEPENDENT', 'WILLFUL_VIOLATOR']]

In [292]:
final_visa_data = pd.get_dummies(final_visa_data, columns= cols)

Finally, I need to convert the CASE_STATUS target column to 1 for certified and 0 for denied. It is crucial that you do no EVER hot encode your target column - do not try to predict a dummy variable or you will become the dummy. 

In [293]:
final_visa_data['CASE_STATUS'] = final_visa_data.CASE_STATUS.map(dict(CERTIFIED= 1, DENIED= 0))

In [297]:
final_visa_data.tail(10)

,CASE_STATUS,TOTAL_WORKERS,NEW_EMPLOYMENT,CONTINUED_EMPLOYMENT,CHANGE_PREVIOUS_EMPLOYMENT,NEW_CONCURRENT_EMPLOYMENT,CHANGE_EMPLOYER,AMENDED_PETITION,PREVAILING_WAGE,WAGE_RATE_OF_PAY_FROM,...,PW_SOURCE_DBA,PW_SOURCE_OES,PW_SOURCE_Other,PW_SOURCE_SCA,WAGE_RATE_OF_PAY_TO_N,WAGE_RATE_OF_PAY_TO_Y,H1B_DEPENDENT_N,H1B_DEPENDENT_Y,WILLFUL_VIOLATOR_N,WILLFUL_VIOLATOR_Y
624616,0,2,2,0,0,0,0,0,68369.60,84600.00,...,0,0,1,0,0,1,1,0,1,0
624618,0,1,0,0,0,0,1,0,79144.00,79144.00,...,0,1,0,0,0,1,1,0,1,0
624642,1,1,0,1,0,0,0,0,49940.80,28.50,...,0,0,1,0,1,0,1,0,1,0
624643,1,1,1,0,0,0,0,0,120348.80,57.86,...,0,0,1,0,1,0,1,0,1,0
624644,1,1,0,0,0,1,0,0,120348.80,57.86,...,0,0,1,0,1,0,1,0,1,0
624645,1,1,0,1,0,0,0,0,77209.60,46.00,...,0,1,0,0,1,0,0,1,1,0
624646,1,1,0,1,0,0,0,0,50398.00,60400.00,...,0,1,0,0,1,0,1,0,1,0
624647,1,1,1,0,0,0,0,0,61069.00,65000.00,...,0,1,0,0,1,0,1,0,1,0
624648,1,1,0,1,0,0,0,0,41392.00,49383.00,...,0,1,0,0,1,0,1,0,1,0
624649,1,1,1,0,0,0,0,0,45198.00,45200.00,...,0,1,0,0,1,0,1,0,1,0


Much better. This data set is ready to transfer to the ubuntu AWS ec2 instance and load into a spark data from for machine learning. 

In [296]:
# write to csv
final_visa_data.to_csv("./Desktop/big data course proj/H-1B data/updated_2017_data.csv", index = False)